In [ ]:
import cv2
import numpy as np

# Skin color range in HSV (works for most lighting; you can tweak later)
lower = np.array([0, 30, 60])
upper = np.array([25, 255, 255])

# Virtual boundary (rectangle)
rect_x1, rect_y1 = 350, 150
rect_x2, rect_y2 = 600, 350

def point_rect_distance(px, py, x1, y1, x2, y2):
    cx = min(max(px, x1), x2)
    cy = min(max(py, y1), y2)
    return int(np.hypot(px - cx, py - cy))

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, lower, upper)

    # Morphological cleanup
    mask = cv2.medianBlur(mask, 7)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    fingertip = None

    if contours:
        cnt = max(contours, key=cv2.contourArea)

        if cv2.contourArea(cnt) > 2000:
            hull = cv2.convexHull(cnt)

            # Find centroid
            M = cv2.moments(cnt)
            if M["m00"] != 0:
                cx = int(M["m10"]/M["m00"])
                cy = int(M["m01"]/M["m00"])
                center = (cx, cy)

                # Fingertip = hull point farthest from centroid
                max_d = 0
                for p in hull:
                    px, py = p[0]
                    d = np.hypot(px - cx, py - cy)
                    if d > max_d:
                        max_d = d
                        fingertip = (px, py)

                if fingertip:
                    cv2.circle(frame, fingertip, 10, (0, 0, 255), -1)

    # Draw boundary rectangle
    cv2.rectangle(frame, (rect_x1, rect_y1), (rect_x2, rect_y2), (255, 255, 255), 2)

    state = "SAFE"

    if fingertip:
        dist = point_rect_distance(fingertip[0], fingertip[1],
                                   rect_x1, rect_y1, rect_x2, rect_y2)

        if dist < 20:
            state = "DANGER"
        elif dist < 80:
            state = "WARNING"
        else:
            state = "SAFE"

        # Draw distance line
        cv2.putText(frame, f"Dist: {dist}", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    # State feedback
    if state == "SAFE":
        color = (0, 255, 0)
    elif state == "WARNING":
        color = (0, 255, 255)
    else:
        color = (0, 0, 255)
        cv2.putText(frame, "DANGER DANGER", (50, 250),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 4)

    cv2.putText(frame, f"STATE: {state}", (10, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

    cv2.imshow("Hand Safety System", frame)
    cv2.imshow("Mask", mask)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
